In [1]:
%load_ext autoreload
%autoreload 2

import astropy
import galsim
import matplotlib.pyplot as plt
import numpy as np
import scarlet
import sep

import btk

# SCARLET implementation

This notebook provides a measure function using [SCARLET](https://www.sciencedirect.com/science/article/abs/pii/S2213133718300301), a deblending algorithm based on matrix factorization. **NOTE:** It requires that you install the scarlet python package from the [source](https://github.com/pmelchior/scarlet), the pip installation being outdated. Please follow the instructions for installing scarlet [here](https://pmelchior.github.io/scarlet/install.html). 

If you have not done so already, we encourage you to follow the BTK [intro tutorial](https://lsstdesc.org/BlendingToolKit/tutorials.html), which will help you understand what is done in this notebook.

In [2]:
catalog_name = "../data/sample_input_catalog.fits"
stamp_size = 24
survey = btk.survey.Rubin


In [3]:
catalog = btk.catalog.CatsimCatalog.from_file(catalog_name)
draw_blend_generator = btk.draw_blends.CatsimGenerator(
    catalog,
    btk.sampling_functions.DefaultSampling(max_number=5,maxshift=6),
    [survey],
    stamp_size=stamp_size,
    batch_size=100
)

In [4]:
def scarlet_measure(batch,idx,channels_last=False,surveys=None,**kwargs):
    """Measure function for SCARLET
    """
    if isinstance(batch["blend_images"], dict):
        raise NotImplementedError("This function does not support the multi-resolution feature.")
    
    sigma_noise = kwargs.get("sigma_noise", 1.5)
    mean_sky_level = [btk.survey.get_mean_sky_level(surveys[0],filt) for filt in surveys[0].filters]
    
    #Fist we carry out the detection, using SExtractor (sep being the python implementation)
    image = batch["blend_images"][idx]
    stamp_size = image.shape[-2] 
    channel_indx = 0 if not channels_last else -1
    coadd = np.mean(image, axis=channel_indx) 
    bkg = sep.Background(coadd)
    catalog, segmentation = sep.extract(
        coadd, sigma_noise, err=bkg.globalrms, segmentation_map=True
    )
    
    # Put the image in the channels first format if not already the case
    # The shape of the image is now (n_channels,stamp_size,stamp_size)
    image = np.moveaxis(image,-1,0) if channels_last else image
    
    #Drawing the PSFs
    psf = np.array([p.drawImage(galsim.Image(image.shape[1],image.shape[2]),scale=survey.pixel_scale).array for p in batch["psf"]])
    
    ### Initializing scarlet ###
    bands=[f.name for f in surveys[0].filters]
    model_psf = scarlet.GaussianPSF(sigma=(0.8,) * len(bands))
    
    # The frame contains informations about the blend
    model_frame = scarlet.Frame(image.shape, psf=model_psf, channels=bands)
    
    # The observation contains the blended images as well as additionnal informations
    observation = scarlet.Observation(
        image, psf=scarlet.ImagePSF(psf), weights=np.ones(image.shape)/ (bkg.globalrms**2), channels=bands
    ).match(model_frame)
    
    # We define a source for each detection
    sources = []
    for n, detection in enumerate(catalog):
        result = scarlet.ExtendedSource(
            model_frame,
            (detection["y"], detection["x"]),
            observation,
            thresh=1,
            shifting=True,
        )
        sources.append(result)
    
    ### Fitting the sources to the blend ###
    try:
        blend = scarlet.Blend(sources, observation)
        blend.fit(200, e_rel=1e-5)
 
        ### Returning the results in a BTK compatible form ###
        im, selected_peaks = [], []
        for k, component in enumerate(sources):
            y, x = component.center
            selected_peaks.append([x, y])
            model = component.get_model(frame=model_frame)
            model_ = observation.render(model)
            model_ = np.transpose(model_, axes=(1, 2, 0)) if channels_last else model_
            im.append(model_)
        selected_peaks = np.array(selected_peaks)
        t = astropy.table.Table()
        t["x_peak"] = selected_peaks[:,0]
        t["y_peak"] = selected_peaks[:,1]
        deblended_images = np.array(im)
    
    except AssertionError: #If the fitting fails
        t = astropy.table.Table()
        t["x_peak"] = catalog["x"]
        t["y_peak"] = catalog["y"]
        deblended_images=None
    
    return {"catalog":t,"segmentation":None,"deblended_images":deblended_images}
    
    

In [5]:
measure_kwargs=[{"sigma_noise": 2.0}]
meas_generator = btk.measure.MeasureGenerator(
    [btk.measure.sep_measure,scarlet_measure], draw_blend_generator, measure_kwargs=measure_kwargs
)
metrics_generator = btk.metrics.MetricsGenerator(
    meas_generator,
    use_metrics=("detection", "reconstruction"),
    target_meas={"ellipticity": btk.metrics.meas_ksb_ellipticity},
)


In [ ]:
blend_results,measure_results,metrics_results = next(metrics_generator)

# Plot Metrics from Scarlet Results

In [ ]:
btk.plot_utils.plot_metrics_summary(metrics_results)